# Crash Attribute Segmentation of New Jersey State Vehicle Accident Data
## Data Cleaning, Processing, and Exploration

This notebook outlines the process of cleaning, processing, and exploring accident data downloaded from the New Jersey Department of Transportation (NJDOT) for 2014 through 2018. The steps of this process are as follows:

1. Data Cleaning in Excel and Python
2. Data Exploration using Visualization
3. Processing Data for Principle Component Analysis

The resulting file of this process is a CSV of county-municipality locations with normalized values for each crash attribute column.

In [1]:
import os
import glob
# data managing and display libs
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib

%matplotlib inline 

## 1. Data Cleaning in Excel and Python
This section explains the data cleaning process using Excel and Python. The raw data downloaded from NJDOT needed to be pre-processed before being fed into a K-Means model. Below is a list of problems found when exploring the data:

1. County and Municipality values were integers instead of readable titles. These integer values would need to be transcribed and concatenated so that rows can be grouped by County-Municipality in a future processing step.
2. Many columns contained textual values, counts, and file meta-data along with crash attribute columns. Many of these columns would need to be removed so that crash attribute columns can be isolated.
3. Some crash attribute columns are retired in (2017-2018) compared to (2014-2016). Values for these retired columns will need to be queried from the other avaliable tables from the NJDOT site.
4. Each row signifies a particular crash accident which means there are multiple data points per County-Municipality. Since this project focuses on finding crash patterns among municipalities, the data will need to be encoded and grouped by municipality.

### Data Cleaning
Problems 1 through 3 were solved using Excel and SQL. A data dictonary containing County-Municipality titles with their associated integer keys was created to transcribe the accident locations. Using the VLOOKUP function in Excel, readable County-Municipality titles were added to each row and columns containing the integer keys were removed. 

Columns which contained textual descriptions, file meta-data, and counts for vehicles or persons were removed manually in Excel. This was done to focus more on crash attributes that may be similar across several municipalities instead of quantitive data which may vary across municipalities.

Columns that were retired or added between years had to be verified or queried for. For example, 2014-2016 data did not have the "first_harmful_event" column which is a value from the Vehicles table avaliable on NJDOT's site. To bridge this gap, SQL was used to locate values from external tables for these columns using the unique "crashid" value and joined to the accidents data.

Problem 4 was solved using Python and is documented below. The accidents data used for this notebook is the pre-processed data after completing Problems 1 through 3.

### Combine 2014-2018 CSVs into one Dataframe
Because the data was seperated by year, the data needs to be combined into a single Dataframe before it is grouped and normalized. Rows that do not have complete data are deleted.

In [4]:
crash_data_csv = pd.DataFrame() # this will hold all accident data from the CSVs

parent_folder = r'C:\Users\Seu\Documents\Udacity\Machine Learning\Capstone\Capstone_data\crash_data_severity'
crash_files = glob.glob(os.path.join(parent_folder, "*.csv"))     # advisable to use os.path.join as this makes concatenation OS independent

crash_data_csv = pd.concat((pd.read_csv(f, header=0, delimiter=",", index_col=None) for f in crash_files))

print(crash_data_csv.shape)
crash_data_csv.head()


(7878, 20)


,crashid,location,severity_rating5,intersection,alcohol_involved,hazmat_involved,hs_collision_type,crash_type,road_sys_code,road_horiz_align_code,road_grade_code,road_surf_code,surf_cond_code,light_cond_code,environ_cond_code,road_median_code,temp_traffic_zone,posted_speed,first_harmful_event,flg_cell_in_use
0,01-02-2014-14-086659,ATLANTIC-ATLANTIC CITY,4,N,N,N,1,13.0,7,1.0,4.0,2.0,1.0,1.0,1.0,2.0,1.0,25.0,22.0,N
1,01-02-2014-14-083519,ATLANTIC-ATLANTIC CITY,4,N,Y,N,1,13.0,7,1.0,4.0,2.0,1.0,6.0,5.0,4.0,1.0,25.0,22.0,N
2,01-02-2014-14-131338,ATLANTIC-ATLANTIC CITY,5,Y,N,N,2,3.0,7,1.0,4.0,2.0,1.0,1.0,1.0,5.0,1.0,25.0,26.0,Y
3,01-02-2014-14-115664,ATLANTIC-ATLANTIC CITY,5,Y,N,N,1,13.0,7,1.0,4.0,1.0,2.0,6.0,2.0,4.0,NaN,25.0,22.0,N
4,01-02-2014-14-099046,ATLANTIC-ATLANTIC CITY,4,N,N,N,1,13.0,7,1.0,4.0,2.0,1.0,1.0,1.0,5.0,1.0,25.0,22.0,N


In [5]:
# drop any incomplete data
crash_data_csv = crash_data_csv.dropna(axis=0)
print('(clean) rows, cols: ', crash_data_csv.shape)

(clean) rows, cols:  (6819, 20)


In [6]:
# save the combined dataframe into a csv for future use
process_folder = r'C:\Users\Seu\Documents\Udacity\Machine Learning\Capstone\Capstone_data\processed_data'
savePath =  os.path.join(process_folder, 'accidents_combined_severity.csv')
crash_data_csv.to_csv(savePath)


## 2. Data Exploration

## 3. Processing Data for Principle Component Analysis
### Encode Columns using One Hot Encoding
Data in the Accidents table consists of rows containing information about one instance of a crash. Information includes the Crash ID, county, municipality, and several crash attributes such as alcohol involvement, road surface conditions, and hazards. The data format for these columns range from Y/N letters to numerical codes, all of which are categorial variables. 

Because this project focuses on exploring crash patterns between municipalities, the current format of the Accidents table cannot be used to train the K-Means model. Instead, the data needs to be grouped by municipality and each categorical column must be converted to a numerical value. One Hot Encoding was used to convert categorial values into one-dimensional vectors. Encoded rows were spotchecked against the CSV data in Excel. Afterwards, each column was summed based on municipality groups then processed in the normalization step.

**A big shoutout to my Proposal reviewer for suggesting this technique to me!!**

In [7]:
def createEncodings(aDataframe):
    encodingDataFrame = pd.DataFrame()
    for (columnName, columnData) in aDataframe.iteritems():
        if columnName in ["location", "crashid"]:
            continue;
        dfDummies = pd.get_dummies(aDataframe[columnName], prefix=columnName)
        encodingDataFrame = pd.concat([encodingDataFrame, dfDummies], axis=1)
    return encodingDataFrame

In [8]:
encoded_columns = createEncodings(crash_data_csv)
encoded_columns.head()

,severity_rating5_4,severity_rating5_5,intersection_N,intersection_Y,alcohol_involved_N,alcohol_involved_Y,hazmat_involved_N,hazmat_involved_Y,hs_collision_type_1,hs_collision_type_2,...,first_harmful_event_57.0,first_harmful_event_58.0,first_harmful_event_59.0,first_harmful_event_60.0,first_harmful_event_61.0,first_harmful_event_62.0,first_harmful_event_69.0,first_harmful_event_99.0,flg_cell_in_use_N,flg_cell_in_use_Y
0,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,1,0,1,0,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
2,0,1,0,1,1,0,1,0,0,1,...,0,0,0,0,0,0,0,0,0,1
4,1,0,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,1,0,1,0,0,1,1,0,0,1,...,0,0,0,0,0,0,0,0,1,0


In [9]:
# append the encoded dataframe to the LOCATION column
encoded_crash_data = pd.DataFrame()
encoded_crash_data = pd.concat([crash_data_csv[["crashid", "location"]].copy(), encoded_columns], axis=1)
encoded_crash_data.head()

,crashid,location,severity_rating5_4,severity_rating5_5,intersection_N,intersection_Y,alcohol_involved_N,alcohol_involved_Y,hazmat_involved_N,hazmat_involved_Y,...,first_harmful_event_57.0,first_harmful_event_58.0,first_harmful_event_59.0,first_harmful_event_60.0,first_harmful_event_61.0,first_harmful_event_62.0,first_harmful_event_69.0,first_harmful_event_99.0,flg_cell_in_use_N,flg_cell_in_use_Y
0,01-02-2014-14-086659,ATLANTIC-ATLANTIC CITY,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
1,01-02-2014-14-083519,ATLANTIC-ATLANTIC CITY,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0
2,01-02-2014-14-131338,ATLANTIC-ATLANTIC CITY,0,1,0,1,1,0,1,0,...,0,0,0,0,0,0,0,0,0,1
4,01-02-2014-14-099046,ATLANTIC-ATLANTIC CITY,1,0,1,0,1,0,1,0,...,0,0,0,0,0,0,0,0,1,0
5,01-02-2014-14-005817,ATLANTIC-ATLANTIC CITY,1,0,1,0,0,1,1,0,...,0,0,0,0,0,0,0,0,1,0


In [10]:
# save the encoded dataframe into a csv for future use
savePath =  os.path.join(process_folder, 'accidents_encoded_severity.csv')
encoded_crash_data.to_csv(savePath)

### Group by location and sum each column
Before normalizing the data, each column needs to be summed by municipality. The resulting sums from this dataframe were checked against the CSV data in Excel using tabulaton tools.

In [11]:
encoded_crash_data.drop(['crashid'], axis=1)
columnNames = encoded_crash_data.columns
crash_data = encoded_crash_data.groupby(['location'])[columnNames].sum()
crash_data.head()

,severity_rating5_4,severity_rating5_5,intersection_N,intersection_Y,alcohol_involved_N,alcohol_involved_Y,hazmat_involved_N,hazmat_involved_Y,hs_collision_type_1,hs_collision_type_2,...,first_harmful_event_57.0,first_harmful_event_58.0,first_harmful_event_59.0,first_harmful_event_60.0,first_harmful_event_61.0,first_harmful_event_62.0,first_harmful_event_69.0,first_harmful_event_99.0,flg_cell_in_use_N,flg_cell_in_use_Y
location,,,,,,,,,,,,,,,,,,,,,
ATLANTIC-ABSECON CITY,4,4,6,2,3,5,8,0,3,0,...,0,0,0,0,0,0,0,0,8,0
ATLANTIC-ATLANTIC CITY,32,6,25,13,28,10,38,0,11,13,...,0,0,0,1,0,0,0,0,37,1
ATLANTIC-BUENA BORO,2,1,2,1,3,0,3,0,0,2,...,0,0,0,0,0,0,0,0,3,0
ATLANTIC-BUENA VISTA TWP,9,6,12,3,13,2,15,0,1,3,...,0,0,0,2,0,0,0,0,15,0
ATLANTIC-EGG HARBOR CITY,5,4,3,6,8,1,9,0,2,4,...,0,0,0,0,0,0,0,0,9,0


In [12]:
# save the grouped dataframe into a csv for future use
savePath =  os.path.join(process_folder, 'accidents_grouped_severity.csv')
crash_data.to_csv(savePath)

### Normalize the data
MinMaxScalar was used to standardize the scale of all crash attribute columns so that they can be consistently compared to the values of the other columns.

In [13]:
# scale numerical features into a normalized range, 0-1
from sklearn.preprocessing import MinMaxScaler

scaler=MinMaxScaler()
# store them in this dataframe
crash_data_scaled=pd.DataFrame(scaler.fit_transform(crash_data.astype(float)))

# get same features and County-Municipality indices
crash_data_scaled.columns=crash_data.columns
crash_data_scaled.index=crash_data.index

crash_data_scaled.head()

,severity_rating5_4,severity_rating5_5,intersection_N,intersection_Y,alcohol_involved_N,alcohol_involved_Y,hazmat_involved_N,hazmat_involved_Y,hs_collision_type_1,hs_collision_type_2,...,first_harmful_event_57.0,first_harmful_event_58.0,first_harmful_event_59.0,first_harmful_event_60.0,first_harmful_event_61.0,first_harmful_event_62.0,first_harmful_event_69.0,first_harmful_event_99.0,flg_cell_in_use_N,flg_cell_in_use_Y
location,,,,,,,,,,,,,,,,,,,,,
ATLANTIC-ABSECON CITY,0.031250,0.066667,0.053571,0.026316,0.017647,0.227273,0.037433,0.0,0.045455,0.000000,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.037838,0.000000
ATLANTIC-ATLANTIC CITY,0.250000,0.100000,0.223214,0.171053,0.164706,0.454545,0.197861,0.0,0.166667,0.206349,...,0.0,0.0,0.0,0.166667,0.0,0.0,0.0,0.0,0.194595,0.333333
ATLANTIC-BUENA BORO,0.015625,0.016667,0.017857,0.013158,0.017647,0.000000,0.010695,0.0,0.000000,0.031746,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.010811,0.000000
ATLANTIC-BUENA VISTA TWP,0.070312,0.100000,0.107143,0.039474,0.076471,0.090909,0.074866,0.0,0.015152,0.047619,...,0.0,0.0,0.0,0.333333,0.0,0.0,0.0,0.0,0.075676,0.000000
ATLANTIC-EGG HARBOR CITY,0.039062,0.066667,0.026786,0.078947,0.047059,0.045455,0.042781,0.0,0.030303,0.063492,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.043243,0.000000


In [14]:
# save the normalized dataframe into a csv for future use
savePath =  os.path.join(process_folder, 'accidents_normalized_severity.csv')
crash_data_scaled.to_csv(savePath)

In [15]:
crash_data_scaled.describe()

,severity_rating5_4,severity_rating5_5,intersection_N,intersection_Y,alcohol_involved_N,alcohol_involved_Y,hazmat_involved_N,hazmat_involved_Y,hs_collision_type_1,hs_collision_type_2,...,first_harmful_event_57.0,first_harmful_event_58.0,first_harmful_event_59.0,first_harmful_event_60.0,first_harmful_event_61.0,first_harmful_event_62.0,first_harmful_event_69.0,first_harmful_event_99.0,flg_cell_in_use_N,flg_cell_in_use_Y
count,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,...,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000,511.000000
mean,0.066353,0.080855,0.084708,0.050752,0.065017,0.104163,0.065814,0.012394,0.035759,0.066070,...,0.025440,0.034247,0.012720,0.082518,0.014351,0.006849,0.030333,0.029354,0.066039,0.042401
std,0.095535,0.119637,0.112974,0.090128,0.094331,0.150884,0.099486,0.075686,0.070825,0.110580,...,0.157613,0.147865,0.075632,0.161646,0.076772,0.066066,0.095536,0.168963,0.100022,0.125880
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.015625,0.016667,0.017857,0.013158,0.017647,0.000000,0.010695,0.000000,0.000000,0.015873,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.010811,0.000000
50%,0.039062,0.033333,0.044643,0.026316,0.035294,0.045455,0.032086,0.000000,0.015152,0.031746,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.032432,0.000000
75%,0.078125,0.100000,0.107143,0.052632,0.076471,0.136364,0.074866,0.000000,0.045455,0.079365,...,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000,0.000000,0.075676,0.000000
max,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000
